In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Dependencies

In [8]:
!uv pip install dotenv "numpy<2.0.0"

Audited 2 packages in 12ms


In [9]:
!uv pip install openpipe-art openpipe --prerelease allow --no-cache-dir

Audited 2 packages in 58ms


# Environment variables

In [10]:
import os

# Optional
WANDB_API_KEY = ""
if WANDB_API_KEY:
    os.environ["WANDB_API_KEY"] = WANDB_API_KEY

# Optional
OPENPIPE_API_KEY = ""
if OPENPIPE_API_KEY:
    os.environ["OPENPIPE_API_KEY"] = OPENPIPE_API_KEY
    
MODEL_NAME = "001"
PROJECT = "generate-fhir-single-turn"
BASE_MODEL = "Qwen/Qwen2.5-7B-Instruct"
LEARNING_RATE = 1.2e-5
GROUPS_PER_STEP = 1
EVAL_STEPS = 50
VAL_SET_SIZE = 100
NUM_EPOCHS = 1
NUM_GENERATIONS = 6

# Main loop

In [ ]:
import datasets
import art
from art.local import LocalBackend
from openpipe import AsyncOpenPipe

backend = LocalBackend()
model = art.TrainableModel(
    name=MODEL_NAME,
    project=PROJECT,
    base_model=BASE_MODEL,
    _internal_config=art.dev.InternalModelConfig(
        init_args=art.dev.InitArgs(
            gpu_memory_utilization=0.75,
        ),
        peft_args=art.dev.PeftArgs(
            lora_alpha=8,
        ),
        trainer_args=art.dev.TrainerArgs(
            max_grad_norm=0.1,
        ),
    ),
)
await model.register(backend)
op_client = AsyncOpenPipe(api_key=os.getenv("OPENPIPE_API_KEY"))

## Load the training data
print("Loading training data...")
train_dataset: datasets.Dataset = datasets.load_dataset("../data/fhir-single-turn")

train_data_list: List[Dict[str, Any]] = list(train_dataset)  # type: ignore
print(f"Training data size: {len(train_data_list)}")

# Get OpenAI Client for the ART Model
openai_client = model.openai_client()

# Training Loop
start_step = await model.get_step()
print(f"Starting training from global step {start_step}")

data_iterator = art.utils.iterate_dataset(
    dataset=train_data_list,
    groups_per_step=GROUPS_PER_STEP,
    num_epochs=NUM_EPOCHS,
    initial_step=start_step,
    use_tqdm=True,
)

for batch_inputs, epoch, global_step, epoch_step in data_iterator:
  print(batch_inputs)
  print(epoch)
  print(global_step)
  print(epoch_step)
  break

MPS available, using Apple Silicon GPU


AssertionError: Torch not compiled with CUDA enabled